# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
city_df = pd.read_csv('output_data/cities.csv')
city_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,New Norfolk,-42.78,147.06,48.00,88,85,1.01,AU,1603826686
1,Kununurra,-15.77,128.73,78.80,94,97,2.93,AU,1603826686
2,Hobart,-42.88,147.33,48.99,93,75,1.12,AU,1603826687
3,Bengkulu,-3.80,102.27,75.65,84,94,4.81,ID,1603826446
4,Dauphin,51.15,-100.05,35.60,80,90,10.29,CA,1603826688
...,...,...,...,...,...,...,...,...,...
541,Japura,-0.32,102.35,74.16,95,99,2.26,ID,1603827011
542,Lakhdenpokh'ya,61.52,30.20,50.00,91,100,12.10,RU,1603827012
543,Aklavik,68.22,-135.01,10.40,85,40,2.24,CA,1603827012
544,Santa Cruz,30.42,-9.60,66.20,77,0,9.17,MA,1603827013


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key = g_key)

In [5]:
locations = city_df[['Lat','Lng']].astype(float)
weight = city_df['Humidity']

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(
    locations,
    weights=weight, 
    dissipating=False,
    max_intensity=100,
    point_radius = 1
)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
ideal_df = city_df.loc[(city_df['Max Temp'] > 80) & (city_df['Max Temp'] < 90) & (city_df['Humidity'] < 70) 
                       & (city_df['Humidity'] > 50) & (city_df['Cloudiness'] < 50),:]
ideal_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
30,Djibouti,11.50,42.50,82.40,58,1,15.05,DJ,1603826703
56,São João da Barra,-21.64,-41.05,82.40,65,0,20.80,BR,1603826722
83,Touros,-5.20,-35.46,84.20,66,20,18.34,BR,1603826737
135,Itarema,-2.92,-39.92,83.26,67,1,15.10,BR,1603826768
156,Cabo San Lucas,22.89,-109.91,86.00,54,20,9.17,MX,1603826780
183,Conceição do Mato Dentro,-19.04,-43.42,83.86,54,25,3.98,BR,1603826796
221,Conde,-7.26,-34.91,82.40,69,40,10.29,BR,1603826820
236,San Cristobal,7.77,-72.22,89.60,55,40,9.17,VE,1603826579
262,Belmonte,-15.86,-38.88,82.40,69,20,16.11,BR,1603826842
322,Gboko,7.32,9.00,82.13,58,0,1.72,NG,1603826874


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
hotel_df = ideal_df.copy()
hotel_df['Hotel Name'] = ''
params = {
    'key': g_key,
    'types': 'lodging',
    'radius': 5000
}

for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    coord = f'{lat}, {lng}'
    
    params['location'] = coord
    
    response = requests.get(base_url, params=params).json()
    
    hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
    
    


{'business_status': 'OPERATIONAL',
 'geometry': {'location': {'lat': 11.5309783, 'lng': 42.504256},
              'viewport': {'northeast': {'lat': 11.5323272802915,
                                         'lng': 42.5056049802915},
                           'southwest': {'lat': 11.5296293197085,
                                         'lng': 42.5029070197085}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
 'name': 'Orchid Main Camp Goubet',
 'place_id': 'ChIJh80UQunLPBYR_K-7DicY8xI',
 'plus_code': {'global_code': '7H34GGJ3+9P'},
 'reference': 'ChIJh80UQunLPBYR_K-7DicY8xI',
 'scope': 'GOOGLE',
 'types': ['lodging', 'point_of_interest', 'establishment'],
 'vicinity': ''}
{'business_status': 'OPERATIONAL',
 'geometry': {'location': {'lat': -21.6408235, 'lng': -41.0521534},
              'viewport': {'northeast': {'lat': -21.6395370197085,
                                         'lng': -41.05054781970851},
                           'southwest

{'business_status': 'OPERATIONAL',
 'geometry': {'location': {'lat': 7.7820315, 'lng': -72.2124438},
              'viewport': {'northeast': {'lat': 7.783368630291502,
                                         'lng': -72.21120656970848},
                           'southwest': {'lat': 7.780670669708497,
                                         'lng': -72.2139045302915}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
 'name': 'Pirineos',
 'photos': [{'height': 3024,
             'html_attributions': ['<a '
                                   'href="https://maps.google.com/maps/contrib/110113618717519744303">Elvys '
                                   'Matos</a>'],
             'photo_reference': 'CmRaAAAA0ytfNGIkmgVFYvX56MCjWktBp2-Ony9-Ub_a0gSQTsIijBEgqGmyTzhDQgcRCMDcyvvwFTJjOdPmtcHRClUbExspHjrvnKh4D7uql7LMV4CEQxVtvEVMSA1zKoLTeCV5EhCtgp0pkmWYeRZKgPgpELeUGhRHuvZgITc1GObl7--ActXdRnqfdg',
             'width': 4032}],
 'place_id': 'ChIJ6zVjsfJsZo4Rpef

{'business_status': 'OPERATIONAL',
 'geometry': {'location': {'lat': 23.768031, 'lng': -98.206294},
              'viewport': {'northeast': {'lat': 23.7693418302915,
                                         'lng': -98.2048802697085},
                           'southwest': {'lat': 23.7666438697085,
                                         'lng': -98.20757823029152}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
 'name': 'Hotel Chicago',
 'photos': [{'height': 2448,
             'html_attributions': ['<a '
                                   'href="https://maps.google.com/maps/contrib/116801871993963910738">Roberto '
                                   'Cantero</a>'],
             'photo_reference': 'CmRaAAAAruNafP2EYjbNHsVFoj5PwBnbVBENqluVTCcXfbX3lWV0CWl8x9CD4O5_5h6Xt00UJcp9CHB5xUdPRX2ExT9T3DZQcsPGlNpSCcfWkW07jKu8rEkUTr-hEW7xUFv9NW4REhAe9aHSPJIe10PAbJvpN46GGhT5SGccD9CWCNOnXZ2UstuYHWNhYg',
             'width': 2448}],
 'place_id': 'ChIJ1_ikSsSNe

In [19]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
30,Djibouti,11.50,42.50,82.40,58,1,15.05,DJ,1603826703,Orchid Main Camp Goubet
56,São João da Barra,-21.64,-41.05,82.40,65,0,20.80,BR,1603826722,Pousada Porto De Canoas
83,Touros,-5.20,-35.46,84.20,66,20,18.34,BR,1603826737,INN NEW HORIZON
135,Itarema,-2.92,-39.92,83.26,67,1,15.10,BR,1603826768,Windvillage
156,Cabo San Lucas,22.89,-109.91,86.00,54,20,9.17,MX,1603826780,Hotel Tesoro Los Cabos
183,Conceição do Mato Dentro,-19.04,-43.42,83.86,54,25,3.98,BR,1603826796,Hotel Vista Inn
221,Conde,-7.26,-34.91,82.40,69,40,10.29,BR,1603826820,Pousada Beija Flor
236,San Cristobal,7.77,-72.22,89.60,55,40,9.17,VE,1603826579,Pirineos
262,Belmonte,-15.86,-38.88,82.40,69,20,16.11,BR,1603826842,Pousada Monte Carmelo O Pão
322,Gboko,7.32,9.00,82.13,58,0,1.72,NG,1603826874,Bays Garden Hotel


In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [21]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content= hotel_info
)


fig = gmaps.figure()
fig.add_layer(hotel_layer)
fig.add_layer(heat_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))